# 《数据采集与清洗》
## 第一次作业内容： 网页抓取
### 具体目标：
+ 用``urllib.requests``库发起一次``get``请求，输出``response``文本信息；
+ 用``Requests``库伪装成火狐浏览器发起一次``post``请求，输出``response``文本信息；
+ 用``Requests``库中的``session``对象发出``get``请求，设置``cookies``并获取，输出获取的``cookies``内容；
+ 实现抓取网页的不去重深度遍历算法，自选合适的种子网站和相关参数，输出结果；
+ 编写抓取网页的广度遍历算法（含去重和不去重），自选合适的种子网站和相关参数，输出结果；
+ (选)将抓取网页的去重深度遍历算法封装成对象（类），并测试。

### 注：
+ 代码要有注释，结果要有分析；
+ 本次作业提交截至时间：2020年3月10日(星期二)；
+ 文件命名规则: 班级号+学号+姓名+作业序号，示例：``1_20188989899_张三_1``；
+ 提交方式：1班发送至邮箱：632994085@qq.com；2班发送至邮箱：786888939@qq.com.

### 问题1：用``urllib.requests``库发起一次``get``请求，输出``response``文本信息。

In [26]:
# 这里编写代码
import urllib.request
response = urllib.request.urlopen('https://www.baidu.com/')

In [27]:
print(response.code)
print(response.read().decode('utf-8'))

200
<html>
<head>
	<script>
		location.replace(location.href.replace("https://","http://"));
	</script>
</head>
<body>
	<noscript><meta http-equiv="refresh" content="0;url=http://www.baidu.com/"></noscript>
</body>
</html>


这里对结果进行分析。

### 用``Requests``库伪装成火狐浏览器发起一次``post``请求，输出``response``文本信息。

In [13]:
# 这里编写代码
import requests
fake_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36'
}  #浏览器f12里面复制的
response = requests.post("https://www.baidu.com",headers = fake_headers)

In [16]:
print(response.status_code)
print(response.content.decode('utf-8'))

200
<!DOCTYPE html>
<!--STATUS OK-->
<html>
<head>
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <meta http-equiv="content-type" content="text/html;charset=utf-8">
    <meta content="always" name="referrer">
    <script src="https://ss1.bdstatic.com/5eN1bjq8AAUYm2zgoY3K/r/www/nocache/imgdata/seErrorRec.js"></script>
    <title>页面不存在_百度搜索</title>
    <style data-for="result">
        body {color: #333; background: #fff; padding: 0; margin: 0; position: relative; min-width: 700px; font-family: arial; font-size: 12px }
        p, form, ol, ul, li, dl, dt, dd, h3 {margin: 0; padding: 0; list-style: none }
        input {padding-top: 0; padding-bottom: 0; -moz-box-sizing: border-box; -webkit-box-sizing: border-box; box-sizing: border-box } img {border: none; }
        .logo {width: 117px; height: 38px; cursor: pointer }
         #wrapper {_zoom: 1 }
        #head {padding-left: 35px; margin-bottom: 20px; width: 900px }
        .fm {clear: both; position: relative; z

这里对结果进行分析。

### 用``Requests``库中的``session``对象发出``get``请求，设置``cookies``并获取，输出获取的``cookies``内容。 

In [41]:
# 这里编写代码
import requests
http_session = requests.session()   #弄一个session的实例
response = http_session.get("https://www.baidu.com")

In [43]:
http_session.cookies   #获取当前的cookies

<RequestsCookieJar[Cookie(version=0, name='BDORZ', value='27315', port=None, port_specified=False, domain='.baidu.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1583552900, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False)]>

In [30]:
print(response.status_code)
print(response.content.decode('utf-8'))

200
<!DOCTYPE html>
<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=https://ss1.bdstatic.com/5eN1bjq8AAUYm2zgoY3K/r/www/cache/bdorz/baidu.min.css><title>百度一下，你就知道</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hidefocus=true src=//www.baidu.com/img/bd_logo1.png width=270 height=129> </div> <form id=form name=f action=//www.baidu.com/s class=fm> <input type=hidden name=bdorz_come value=1> <input type=hidden name=ie value=utf-8> <input type=hidden name=f value=8> <input type=hidden name=rsv_bp value=1> <input type=hidden name=rsv_idx value=1> <input type=hidden name=tn value=baidu><span class="bg s_ipt_wr"><input id=kw name=wd class=s_ipt value maxlength=255 autocomplete=off autofocus=autofocus></span><span class="bg s

这里对结果进行分析。

### 实现抓取网页的不去重深度遍历算法，自选合适的种子网站和相关参数，输出结果。


In [35]:
# 这里编写代码
import requests
import re
init_url = "https://cn.hao123.com/"
r = re.compile(r'href=[\'"]?(http[^\'" >]+)')
stack = [init_url]
maxnum = 5  #设置最大爬取限制
used =[]   # 保存访问的网址

while len(stack) > 0 and maxnum > 0: #利用深度优先遍历找出所有url（不去重）
    try:
        url = stack.pop(-1)             #这二步就与maxnum挂钩了
        html = requests.get(url).text         
        used.append(url)               # 将已经抓取过的URL存入used集合中
        new_urls = r.findall(html)     # re的方法，将该网站上所有的有关于http开头的url找到并作为列表存储
        for new_url in new_urls:      # 将该网页上抓取的URL添加到stack中
            if new_url not in stack:  # 栈里面已经存在的    
                stack.append(new_url)
        maxnum -= 1
    except Exception as e:
        print(url)
        print(e)
        maxnum -= 1
    

In [36]:
print(used)

['https://cn.hao123.com/', 'https://haokan.baidu.com/v?vid=6950194982662147532&amp;fr=hao123-op', 'https://baijiahao.baidu.com/builder/app/choosetype?src_from=haokanPC', 'https://sv.bdstatic.com/static/haokan-pc/style/videoland.02cae0.css', 'https://sv.bdstatic.com/static/haokan-pc/style/1.ddbde4.css']


这里对结果进行分析。

### 编写抓取网页的广度遍历算法（含去重和不去重），自选合适的种子网站和相关参数，输出结果。 

In [39]:
# 这里编写代码（去重）
import requests
import re
init_url = "https://cn.hao123.com/"
r = re.compile(r'href=[\'"]?(http[^\'" >]+)')
queue = [init_url]
used = set() # 利用hash消除已重复链接
maxnum = 5

while len(queue) > 0 and maxnum > 0: #利用广度优先遍历找出所有不重复url
    try:
        url = queue.pop(0)             #这二步就与maxnum挂钩了    与深度优先遍历的区别就在于栈于队列
        html = requests.get(url).text         
        used.add(url) # 将已经抓取过的URL存入used集合中
        new_urls = r.findall(html)     # re的方法，将该网站上所有的有关于http开头的url找到并作为列表存储
        for new_url in new_urls:      # 将该网页上新发新未抓取的URL添加到queue中
            if new_url not in used and new_url not in queue:       # 还要注意队列里的缓存数据也要加以判断
                queue.append(new_url)
        maxnum -= 1
    except Exception as e:
        print(url)
        print(e)
        maxnum -= 1

In [37]:
# 这里编写代码（不去重）
import requests
import re
init_url = "https://cn.hao123.com/"
r = re.compile(r'href=[\'"]?(http[^\'" >]+)')
queue = [init_url]
used = []
maxnum = 5

while len(queue) > 0 and maxnum > 0: #利用广度优先遍历找出所有url
    try:
        url = queue.pop(0)             #这二步就与maxnum挂钩了   
        html = requests.get(url).text         
        used.append(url) # 将已经抓取过的URL存入used集合中
        new_urls = r.findall(html)     # re的方法，将该网站上所有的有关于http开头的url找到并作为列表存储
        for new_url in new_urls:      # 将该网页上新发新未抓取的URL添加到queue中
            if new_url not in queue:       # 队列的缓存数据就不再存储了
                queue.append(new_url)
        maxnum -= 1
    except Exception as e:
        print(url)
        print(e)
        maxnum -= 1

In [40]:
print(used)

{'https://cn.hao123.com/', 'https://dgss2.bdstatic.com', 'https://dgss0.bdstatic.com', 'https://dgss3.bdstatic.com', 'https://dgss1.bdstatic.com'}


这里对结果进行分析。

### (选)将抓取网页的去重深度遍历算法封装成对象（类），并测试。 

In [31]:
# 这里编写代码
import requests
import re
class Climb_web:
    """
    3月5日新编读取url
    """
    def __init__(self,url):
        self.url = url
    
    def get_url_html(self,fake_headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36'
    } ):

        response = requests.get(self.url,headers=fake_headers)
        return response.content.decode('utf-8')
        
    def get_all_url(self,maxnum): 
        """
        得到网页上指定规模内以href为标签，http开头的网址（广度优先遍历）
        """
        r = re.compile(r'href=[\'"]?(http[^\'" >]+)')
        queue = [self.url]
        used = set() # 利用hash消除已重复链接

        while len(queue) > 0 and maxnum > 0: #利用广度优先遍历找出所有不重复url
            try:
                url = queue.pop(0)             #这二步就与maxnum挂钩了
                html = requests.get(url).text         
                used.add(url) # 将已经抓取过的URL存入used集合中
                new_urls = r.findall(html)     # re的方法，将该网站上所有的有关于http开头的url找到并作为列表存储
                for new_url in new_urls:      # 将该网页上新发新未抓取的URL添加到queue中
                    if new_url not in used and new_url not in queue:       # 还要注意队列里的缓存数据也要加以判断
                        queue.append(new_url)
                maxnum -= 1
            except Exception as e:
                print(url)
                print(e)
                maxnum -= 1
        return list(used)

In [32]:
# 测试
a = Climb_web("https://cn.hao123.com/")
a.get_all_url(2)

['https://dgss0.bdstatic.com', 'https://cn.hao123.com/']

这里对结果进行分析。